In [1]:
getwd()
library("data.table")
library("tm")
library("topicmodels")

[1] "/work/cloud-user/russia_finland/finland_russia"

Loading required package: NLP


In [2]:
integrumreg <- fread("/work/cloud-user/russia_finland/finland_russia/data/processed/integrum_local.csv")



In [7]:
head(integrumreg)

id,date,MediaDetail,MediaShort,headline,content,lemmas_headline,lemmas_content,tokens_content,tokens_headline,year


In [3]:
#create a corpus using lemmarized text
corp <- Corpus(VectorSource(integrumreg$lemmas_content))

In [5]:
## transform to low - OK
corp1 <- tm_map(corp, content_transformer(tolower))




In [17]:
ls()


[1] "corp"        "corp1"       "corp2"       "corp3"       "corp4"      
 [6] "corp5"       "corp6"       "corpvalmis"  "integrumreg" "toSpace"

In [7]:
## remove punctuation - OK
corp2 <- tm_map(corp1, removePunctuation)


In [9]:
## remove other things - very important, /n are breaks in the text, we dont need them
toSpace <- content_transformer(function (x , pattern ) gsub(pattern, " ", x))
corp3 <- tm_map(corp2, toSpace, "\n")



In [11]:
## get rid of white space - do it
corp4 <- tm_map(corp3, stripWhitespace )




In [25]:
## ## remove stop words - here I add to general list my own words
##
darianstopwords <- c(stopwords("russian"), "quot","nbsp","/n")

ls()
head(darianstopwords)


[1] "corp"            "corp1"           "corp2"           "corp3"          
 [5] "corp4"           "corp5"           "corp6"           "corpdtm"        
 [9] "corpLDA10"       "corpvalmis"      "darianstopwords" "integrumreg"    
[13] "LDA10terms"      "toSpace"

[1] "и"   "в"   "во"  "не"  "что" "он"

In [26]:
corp5 <- tm_map(corp4, removeWords, darianstopwords)

In [27]:
## remove numbers - OK 
##NB: this can be a problem, esp. because of ST1 -
##we will loose this company name when removing numbers
##so this time I skipped this step

corp6 <- tm_map(corp5, removeNumbers)




In [30]:
## finalizing: dunno why, but seems like you have to transform it in this way
corpvalmis <- tm_map(corp5, PlainTextDocument)



In [32]:
##debugging - once again remind R that it is a corpus...
#corpvalmis <- Corpus(VectorSource(corpvalmis))

In [33]:
corpdtm <- DocumentTermMatrix(corpvalmis)


In [34]:
corpLDA30 <- LDA(corpdtm, 30)

In [35]:
LDA30terms <- terms(corpLDA30, 30)

In [37]:
head(LDA30terms)

Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,⋯,Topic 21,Topic 22,Topic 23,Topic 24,Topic 25,Topic 26,Topic 27,Topic 28,Topic 29,Topic 30
год,это,год,который,который,год,год,год,год,год,⋯,год,год,год,становиться,год,год,год,год,который,год
финляндия,россия,который,россия,год,который,это,это,который,это,⋯,который,который,который,компания,который,россия,который,это,россия,это
проект,год,свой,год,свой,это,который,который,город,свой,⋯,петербург,это,наш,свой,россия,свой,россия,петербург,петербург,который
финский,новый,это,город,становиться,россия,карелия,наш,наш,карелия,⋯,время,компания,компания,финляндия,это,город,это,карелия,–n,становиться
свой,мир,компания,свой,наш,свой,новый,театр,компания,становиться,⋯,компания,карелия,первый,первый,город,проект,карелия,финляндия,проект,время
карелия,город,петербург,наш,город,время,становиться,россия,новый,район,⋯,работа,проект,карелия,район,время,петербург,город,свой,становиться,первый


In [21]:
write.csv2(LDA10terms, file="LDA10.csv")

In [6]:
#Integrum federal subcorpus
federal_processed <- textProcessor(
  articlesintegrumfed$lemmas_content, 
  metadata = articlesintegrumfed,
  stem=FALSE,
  customstopwords = darianstopwords)

federal_out <- prepDocuments(
  federal_processed$documents, 
  federal_processed$vocab,
  federal_processed$meta,
  lower.thresh = 15)

federal_model_fit <- stm(
  federal_out$documents, 
  federal_out$vocab, 
  K = 30,
  prevalence =~ year, 
  max.em.its = 100,
  data = energy_out$meta, 
  init.type = "Spectral")

ERROR: Error in textProcessor(articlesintegrumfed$lemmas_content, metadata = articlesintegrumfed, : object 'articlesintegrumfed' not found


In [11]:
install.packages("topicmodels")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
Warning message in install.packages("topicmodels"):
“'lib = "/usr/local/lib/R/site-library"' is not writable”

ERROR: Error in install.packages("topicmodels"): unable to install packages


In [ ]:


## LDA in an empirical fashion starts with trying different number of topic



In [ ]:
corpLDA20 <- LDA(corpdtm, 20)
corpLDA30 <- LDA(corpdtm, 30)
corpLDA50 <- LDA(corpdtm, 50)ls()

In [ ]:
##Gibbs is a sampling method - we can use it to improve the result as it will create
##different seed (i.e., start from a different point)
##corpLDA10g <- LDA(corpdtm, 10, method="Gibbs")
##corpLDA20g <- LDA(corpdtm, 20, , method="Gibbs")


## getting terms into CSV - a two-step procedure
## first writing terms into an intermediary string and next that into csv
LDA10terms <- terms(corpLDA10, 30)
LDA20terms <- terms(corpLDA20, 30)
LDA30terms <- terms(corpLDA30, 30)
LDA50terms <- terms(corpLDA50, 30)

In [ ]:

write.csv2(LDA20terms, file="LDA20.csv")
write.csv2(LDA30terms, file="LDA30.csv")
write.csv2(LDA50terms, file="LDA50.csv")